# Solubility Regression

### Yonsei App.Stat.
### Sunwoo Kim

각 분자구조의 융해성(solubility)을 회귀하는 문제입니다.  
사용한 모델은 Message Passing Neural Network입니다.  


In [1]:
import numpy as np
import pandas as pd
from pysmiles import read_smiles
import networkx as nx
from tqdm import tqdm

# Data related
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader

# Model importing
from solubility_gnn import *

# Ignore Warnings
import warnings
warnings.filterwarnings(action='ignore') 

# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
raw_data = pd.read_csv("solubility_original_data.csv")

smile_codes = raw_data.smiles.values
ys = raw_data.iloc[:, 8].values

### Main Data Generating

Smile 코드로 되어있는 분자구조를 networkx의 형식에 맞게 변형한 후, torch geometric의 형태에 맞춰줍니다.

In [3]:
chemical_indexs = {"Br" : 0, 
                  "C" : 1, 
                  "Cl" : 2, 
                  "F" : 3, 
                  "I" : 4, 
                  "N" : 5, 
                  "O" : 6, 
                  "P" : 7, 
                  "S" : 8}
## There are 9 types of chemical indexs. Thus molecular feature should have 9-dimensional vectors.
## There are [1.0, 1.5, 2.0, 3.0] types of bond. Thus edge attribute has 4-dimensional vectors, 

data = []
for i in tqdm(range(smile_codes.shape[0])) : 
    tmp_y = ys[i]
    formula = smile_codes[i]
    mol = read_smiles(formula)
    node_x = torch.zeros((len(nx.get_node_attributes(mol, name = "element")), 
                         9), dtype = torch.float)
    edge_indexs = torch.tensor([[0], [0]], dtype = torch.long)
    edge_attrs = torch.zeros((1,4), dtype = torch.long)
    for x_ in range(node_x.shape[0]) : 
        item_ = nx.get_node_attributes(mol, name = "element")[x_]
        node_x[x_, chemical_indexs[item_]] = 1
        
    ## Edge type 1.0 
    index1 = torch.tensor([np.where(nx.to_numpy_matrix(mol, weight='order') == 1.0)[0],
                           np.where(nx.to_numpy_matrix(mol, weight='order') == 1.0)[1]], dtype=torch.long)
    if index1.shape[1] > 0 : 
        x1 = torch.zeros((index1.shape[1], 4), dtype = torch.float)
        x1[:, 1] = 1
        edge_indexs = torch.hstack([edge_indexs, index1])
        edge_attrs = torch.vstack([edge_attrs, x1])
        
    ## Edge type 1.5
    index1 = torch.tensor([np.where(nx.to_numpy_matrix(mol, weight='order') == 1.5)[0],
                           np.where(nx.to_numpy_matrix(mol, weight='order') == 1.5)[1]], dtype=torch.long)
    if index1.shape[1] > 0 : 
        x1 = torch.zeros((index1.shape[1], 4), dtype = torch.float)
        x1[:, 1] = 1
        edge_indexs = torch.hstack([edge_indexs, index1])
        edge_attrs = torch.vstack([edge_attrs, x1])
        
    ## Edge type 2.0
    index1 = torch.tensor([np.where(nx.to_numpy_matrix(mol, weight='order') == 2.0)[0],
                           np.where(nx.to_numpy_matrix(mol, weight='order') == 2.0)[1]], dtype=torch.long)
    if index1.shape[1] > 0 : 
        x1 = torch.zeros((index1.shape[1], 4), dtype = torch.float)
        x1[:, 1] = 1
        edge_indexs = torch.hstack([edge_indexs, index1])
        edge_attrs = torch.vstack([edge_attrs, x1])
        
    ## Edge type 3.0
    index1 = torch.tensor([np.where(nx.to_numpy_matrix(mol, weight='order') == 3.0)[0],
                           np.where(nx.to_numpy_matrix(mol, weight='order') == 3.0)[1]], dtype=torch.long)
    if index1.shape[1] > 0 : 
        x1 = torch.zeros((index1.shape[1], 4), dtype = torch.float)
        x1[:, 1] = 1
        edge_indexs = torch.hstack([edge_indexs, index1])
        edge_attrs = torch.vstack([edge_attrs, x1])
    
    tmp_data = Data(x = node_x, 
                    edge_index = edge_indexs[:, 1:],
                    edge_attr = edge_attrs[1:, :], 
                    y = torch.tensor([tmp_y], dtype = torch.float))
    
    data.append(tmp_data)

 10%|███████▉                                                                     | 117/1128 [00:00<00:00, 1169.74it/s]E/Z stereochemical information, which is specified by "/", will be discarded
E/Z stereochemical information, which is specified by "/", will be discarded
E/Z stereochemical information, which is specified by "/", will be discarded
E/Z stereochemical information, which is specified by "\", will be discarded
 21%|████████████████                                                             | 235/1128 [00:00<00:00, 1175.62it/s]E/Z stereochemical information, which is specified by "/", will be discarded
E/Z stereochemical information, which is specified by "\", will be discarded
 31%|████████████████████████                                                     | 353/1128 [00:00<00:00, 1161.58it/s]E/Z stereochemical information, which is specified by "/", will be discarded
E/Z stereochemical information, which is specified by "/", will be discarded
 63%|██████████████████████

read_smile에서 지원하지 않는 일부 분자식은 호환되지 않은 것 같습니다.  
데이터가 준비되었으니 train/test split을 수행하고 regression을 수행해보겠습니다.

In [4]:
train_module = DataLoader(data[:700], batch_size = 5)
test_data = data[700:]

### Training

중간중간 validation 데이터에 대해 성능평가를 진행하는 함수입니다.

In [5]:
def evaluator(GNN_model, test_list) : 
    test_loss = 0
    GNN_model.eval()
    for part_d in test_data : 
        part_d = part_d.to(device)
        y_ = GNN_model(part_d, training_with_batch = False)
        loss_ = ((y_ - part_d.y)**2).to("cpu").item()
        test_loss += loss_
    return test_loss/428

In [6]:
model = NMP_Conv(dataset = data[0], 
                latent_dim = [16, 16, 16, 16],
                device = device)

In [7]:
model

NMP_Conv(
  (convs): ModuleList(
    (0): NNConv(9, 16)
    (1): NNConv(16, 16)
    (2): NNConv(16, 16)
    (3): NNConv(16, 16)
  )
  (edge_linear): ModuleList(
    (0): Linear(in_features=4, out_features=144, bias=True)
    (1): Linear(in_features=4, out_features=256, bias=True)
    (2): Linear(in_features=4, out_features=256, bias=True)
    (3): Linear(in_features=4, out_features=256, bias=True)
  )
  (last_linear): Linear(in_features=16, out_features=1, bias=True)
)

In [8]:
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
model.to(device)
criterion = torch.nn.MSELoss()

model.train()
for epoch in range(500):
    epoch_loss = 0
    for data in train_module : 
        data.to(device)
        optimizer.zero_grad()
        out = model(data = data, 
                   training_with_batch = True)
        loss = criterion(out.view(-1), data.y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.to("cpu").detach().item()
    
    if epoch % 10 == 0 : 
        print("Current Epoch {0} / Training Loss {1} / Test Loss {2}".format(epoch, 
                                                                             epoch_loss/700, 
                                                                             evaluator(model, test_data)))

Current Epoch 0 / Training Loss 60.405122839553016 / Test Loss 14.06145891772064
Current Epoch 10 / Training Loss 0.5672759013676217 / Test Loss 2.9552547900524138
Current Epoch 20 / Training Loss 0.4924173589210425 / Test Loss 2.1455987956031257
Current Epoch 30 / Training Loss 0.334241079049451 / Test Loss 1.4925726335443945
Current Epoch 40 / Training Loss 0.2839846223167011 / Test Loss 1.4805337942837418
Current Epoch 50 / Training Loss 0.19278290926877942 / Test Loss 1.2115028605720903
Current Epoch 60 / Training Loss 0.18261436280395305 / Test Loss 1.234845954387699
Current Epoch 70 / Training Loss 0.13969181974551506 / Test Loss 1.0407097304591695
Current Epoch 80 / Training Loss 0.12937329895794392 / Test Loss 1.2184098831131582
Current Epoch 90 / Training Loss 0.11010864088577883 / Test Loss 1.1203683731750513
Current Epoch 100 / Training Loss 0.09735008151137403 / Test Loss 1.0527366639185012
Current Epoch 110 / Training Loss 0.10218576756439039 / Test Loss 1.0559937198910716

이렇게 Regression Task도 Classification과 유사하게 구현 가능합니다.  
이후 다른 모델들도 이용하여 모델성능을 비교해보는 기회를 가져보겠습니다.